### Handling A Long Context via `TransformChatHistory`

This notebook illustrates how you can use the `TransformChatHistory` capability to give any `Conversable` agent an ability to handle a long context. 

In [1]:
## Uncomment to install pyautogen if you don't have it already
#! pip install pyautogen

In [2]:
import autogen
from autogen.agentchat.contrib.capabilities import context_handling

To add this ability to any agent, define the capability and then use `add_to_agent`.

In [3]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": [{"model": "gpt-3.5-turbo", "api_key": "YOUR_API_KEY"}],
    },
)

# Instantiate the capability to manage chat history
manage_chat_history = context_handling.TransformChatHistory(max_tokens_per_message=50, max_messages=10, max_tokens=1000)
# Add the capability to the assistant
manage_chat_history.add_to_agent(assistant)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=10,
)

user_proxy.initiate_chat(assistant, message="plot and save a graph of x^2 from -10 to 10")

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------


assistant (to user_proxy):

Here's a Python code snippet to plot and save a graph of x^2 from -10 to 10:

```python
import matplotlib.pyplot as plt
import numpy as np

# Generate x values from -10 to 10
x = np.linspace(-10, 10, 100)

# Evaluate y values using x^2
y = x**2

# Plot the graph
plt.plot(x, y)

# Set labels and title
plt.xlabel('x')
plt.ylabel('y')
plt.title('Graph of y = x^2')

# Save the graph as an image file (e.g., PNG)
plt.savefig('x_squared_graph.png')

# Show the graph
plt.show()
```

Please make sure to have the `matplotlib` library installed in your Python environment. After executing the code, the graph will be saved as "x_squared_graph.png" in the current directory. You can change the filename if desired.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Figure(640x480)


---------------------

## Why is this important?
This capability is especially useful if you expect the agent histories to become exceptionally large and exceed the context length offered by your LLM.
For example, in the example below, we will define two agents -- one without this ability and one with this ability.

The agent with this ability will be able to handle longer chat history without crashing.

In [4]:
assistant_base = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": [{"model": "gpt-3.5-turbo", "api_key": "YOUR_API_KEY"}],
    },
)

assistant_with_context_handling = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": [{"model": "gpt-3.5-turbo", "api_key": "YOUR_API_KEY"}],
    },
)
# suppose this capability is not available
manage_chat_history = context_handling.TransformChatHistory(max_tokens_per_message=50, max_messages=10, max_tokens=1000)
manage_chat_history.add_to_agent(assistant_with_context_handling)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=2,
)

# suppose the chat history is large
# Create a very long chat history that is bound to cause a crash
# for gpt 3.5
long_history = []
for i in range(1000):
    # define a fake, very long message
    assitant_msg = {"role": "assistant", "content": "test " * 1000}
    user_msg = {"role": "user", "content": ""}

    assistant_base.send(assitant_msg, user_proxy, request_reply=False, silent=True)
    assistant_with_context_handling.send(assitant_msg, user_proxy, request_reply=False, silent=True)
    user_proxy.send(user_msg, assistant_base, request_reply=False, silent=True)
    user_proxy.send(user_msg, assistant_with_context_handling, request_reply=False, silent=True)

try:
    user_proxy.initiate_chat(assistant_base, message="plot and save a graph of x^2 from -10 to 10", clear_history=False)
except Exception as e:
    print("Encountered an error with the base assistant")
    print(e)
    print("\n\n")

try:
    user_proxy.initiate_chat(
        assistant_with_context_handling, message="plot and save a graph of x^2 from -10 to 10", clear_history=False
    )
except Exception as e:
    print(e)

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
Encountered an error with the base assistant
Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 1009487 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}



user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
Truncated 1991 messages.
Truncated 49800 tokens.
assistant (to user_proxy):

Here is the Python code to plot and save a graph of x^2 from -10 to 10:

```python
import matplotlib.pyplot as plt
import numpy as np

# Generate values for x from -10 to 10
x = np.linspace(-10, 10, 100)

# Calculate y values for x^2
y = x ** 2

# Plot the graph
plt.plot(x, y)
plt.xlabel(